In [1]:
%load_ext autoreload
%autoreload now

In [2]:
# os.environ["CUDA_VISIBLE_DEVICES"] = "MIG-08137aa2-e69b-5e74-8390-7997329b1336"

import torch

# Download and convert data

In [3]:
from tqdm import tqdm

from document_segmentation.pagexml.generale_missiven import GeneraleMissiven
from document_segmentation.settings import (
    GENERALE_MISSIVEN_DOCUMENT_DIR,
    GENERALE_MISSIVEN_SHEET,
)

N = None

GENERALE_MISSIVEN_DOCUMENT_DIR.mkdir(parents=True, exist_ok=True)

sheet = GeneraleMissiven(GENERALE_MISSIVEN_SHEET)

existing_docs = {
    path.stem
    for path in GENERALE_MISSIVEN_DOCUMENT_DIR.glob("*.json")
    if path.is_file()
}

for document in tqdm(
    sheet.to_documents(n=N, skip_ids=existing_docs),
    total=(N or len(sheet)) - len(existing_docs),
    desc="Writing documents",
    unit="doc",
):
    document_file = GENERALE_MISSIVEN_DOCUMENT_DIR / f"{document.id}.json"

    with document_file.open("xt") as f:
        f.write(document.model_dump_json())
        f.write("\n")

Writing documents:   0%|          | 0/5 [00:00<?, ?doc/s]

Skipping row with inventory number 1171 due to status message: 'Niet gedigitaliseerd.'
Skipping row with inventory number 2770 due to status message: 'Niet gedigitaliseerd.'
Skipping row with inventory number 2770 due to status message: 'Niet gedigitaliseerd.'
Skipping row with inventory number 2770 due to status message: 'Niet gedigitaliseerd.'
Skipping row with inventory number 2911 due to status message: 'Niet gedigitaliseerd.'


# Load Data

In [4]:
%autoreload now

In [5]:
TRAINING_DATA = 0.8

In [6]:
from document_segmentation.model.dataset import PageDataset
from document_segmentation.settings import MIN_REGION_TEXT_LENGTH

dataset = (
    PageDataset.from_dir(GENERALE_MISSIVEN_DOCUMENT_DIR)
    .remove_short_regions(MIN_REGION_TEXT_LENGTH)
    .shuffle()
)
# TODO: move region filtering to training/classification step?

len(dataset)

Reading JSON files: 100%|██████████| 909/909 [01:07<00:00, 13.44file/s]


191146

In [7]:
dataset._class_counts()

Counter({<Label.IN: 2>: 189343, <Label.BEGIN: 1>: 905, <Label.END: 3>: 898})

In [8]:
dataset.class_weights()

[210.97792494481237, 1.0095170694608755, 212.6206896551724, 191146.0]

In [9]:
dataset[5000]

Page(label=<Label.IN: 2>, regions=[Region(id='region_c50d90ce-7c08-4c50-b52c-0f17352866c8_2', types=(<RegionType.PARAGRAPH: 'paragraph'>, <RegionType.PHYSICAL_STRUCTURE_DOC: 'physical_structure_doc'>, <RegionType.TEXT_REGION: 'text_region'>, <RegionType.PAGEXML_DOC: 'pagexml_doc'>), coordinates=((657, 1139), (645, 1160), (648, 1178), (690, 1196), (750, 1205), (756, 1211), (747, 1233), (732, 1242), (657, 1263), (603, 1272), (566, 1296), (560, 1317), (587, 1329), (618, 1326), (747, 1341), (783, 1353), (801, 1374), (789, 1401), (762, 1419), (771, 1447), (765, 1489), (750, 1525), (726, 1558), (741, 1567), (753, 1585), (726, 1636), (717, 1670), (717, 1724), (726, 1745), (723, 1838), (747, 1905), (759, 1995), (759, 2164), (777, 2248), (783, 2354), (789, 2375), (789, 2471), (798, 2489), (816, 2498), (925, 2492), (1030, 2501), (1085, 2498), (1145, 2507), (1449, 2507), (1530, 2531), (1567, 2531), (1624, 2513), (1762, 2495), (1829, 2498), (2181, 2489), (2422, 2498), (2443, 2489), (2458, 2474), (

In [10]:
split = int(len(dataset) * TRAINING_DATA)

training_data = dataset[:split]
test_data = dataset[split:]

In [11]:
training_data._class_counts()

Counter({<Label.IN: 2>: 151475, <Label.BEGIN: 1>: 726, <Label.END: 3>: 715})

# Train Model

In [12]:
BATCH_SIZE = 32
EPOCHS = 10
# WEIGHTS = torch.Tensor(dataset.class_weights())   # For an imbalanced dataset
WEIGHTS = None  # For a balanced dataset

TRAINING_DATA_MAX = None

In [13]:
%autoreload now

In [14]:
from document_segmentation.pagexml.datamodel.label import Label

sample_size = training_data._class_counts()[Label.BEGIN]
sample_size

726

In [15]:
from document_segmentation.model.page_classifier import PageClassifier

model = PageClassifier()
model

/Users/carstenschnober/LAHTeR/workspace/document-segmentation/.venv/lib/python3.11/site-packages/torch/nn/modules/rnn.py:83: UserWarning: dropout option adds dropout after all but last recurrent layer, so non-zero dropout expects num_layers greater than 1, but got dropout=0.1 and num_layers=1
  warnings.warn("dropout option adds dropout after all but last "


PageClassifier(
  (_embedding): PageEmbedding(
    (_region_model): RegionEmbedding(
      (_transformer_model): RobertaModel(
        (embeddings): RobertaEmbeddings(
          (word_embeddings): Embedding(42774, 768, padding_idx=1)
          (position_embeddings): Embedding(514, 768, padding_idx=1)
          (token_type_embeddings): Embedding(1, 768)
          (LayerNorm): LayerNorm((768,), eps=1e-05, elementwise_affine=True)
          (dropout): Dropout(p=0.1, inplace=False)
        )
        (encoder): RobertaEncoder(
          (layer): ModuleList(
            (0-11): 12 x RobertaLayer(
              (attention): RobertaAttention(
                (self): RobertaSelfAttention(
                  (query): Linear(in_features=768, out_features=768, bias=True)
                  (key): Linear(in_features=768, out_features=768, bias=True)
                  (value): Linear(in_features=768, out_features=768, bias=True)
                  (dropout): Dropout(p=0.1, inplace=False)
              

In [16]:
model._device

'mps'

In [17]:
if model._device == "mps":
    torch.mps.empty_cache()
    print(torch.mps.current_allocated_memory() / 1024**2)

454.147705078125


In [18]:
model.train_(training_data.balance(sample_size).shuffle(), EPOCHS, BATCH_SIZE, WEIGHTS)

 99%|█████████▉| 67/67.71875 [03:29<00:02,  3.19s/batch]/Users/carstenschnober/LAHTeR/workspace/document-segmentation/.venv/lib/python3.11/site-packages/tqdm/std.py:636: TqdmWarning: clamping frac to range [0, 1]
  full_bar = Bar(frac,
100%|██████████| 68/67.71875 [03:31<00:00,  3.11s/batch]


Current allocated memory (MPS): 13141 MB
Driver allocated memory (MPS): 14500 MB
[Loss:	1.158]


100%|██████████| 68/67.71875 [00:06<00:00, 11.22batch/s]


Current allocated memory (MPS): 13141 MB
Driver allocated memory (MPS): 14366 MB
[Loss:	1.068]


100%|██████████| 68/67.71875 [00:06<00:00, 10.06batch/s]


...

Current allocated memory (MPS): 13141 MB
Driver allocated memory (MPS): 14366 MB
[Loss:	0.848]


100%|██████████| 68/67.71875 [00:07<00:00,  9.04batch/s]


Current allocated memory (MPS): 13141 MB
Driver allocated memory (MPS): 14366 MB
[Loss:	0.893]


100%|██████████| 68/67.71875 [00:07<00:00,  8.86batch/s]

Current allocated memory (MPS): 13141 MB
Driver allocated memory (MPS): 14366 MB
[Loss:	0.871]


# Evaluation

In [19]:
import csv
import sys

from torcheval.metrics import (
    MulticlassAccuracy,
    MulticlassF1Score,
    MulticlassPrecision,
    MulticlassRecall,
)
from tqdm import tqdm

from document_segmentation.pagexml.datamodel.label import Label

writer = csv.DictWriter(
    sys.stdout,
    fieldnames=("Predicted", "Actual", "Page ID", "Text", "Scores"),
    delimiter="\t",
)

writer.writeheader()

accuracy = MulticlassAccuracy(num_classes=len(Label))
precision = MulticlassPrecision(average=None, num_classes=len(Label))
recall = MulticlassRecall(average=None, num_classes=len(Label))
f1_score = MulticlassF1Score(average=None, num_classes=len(Label))

for batch in tqdm(
    test_data[:1000].batches(BATCH_SIZE),
    total=len(test_data) / BATCH_SIZE,
    unit="batch",
):
    predicted = model(batch)
    labels = batch.labels()

    _labels = torch.Tensor([label.value - 1 for label in labels]).to(int)
    accuracy.update(predicted, _labels)
    precision.update(predicted, _labels)
    recall.update(predicted, _labels)
    f1_score.update(predicted, _labels)

    for page, pred, label in zip(batch.pages, predicted, labels):
        pred_label = Label(pred.argmax().item() + 1)
        if pred_label != Label.IN or label != Label.IN:
            writer.writerow(
                {
                    "Predicted": pred_label.name,
                    "Actual": label.name,
                    "Page ID": page.doc_id,
                    "Text": page.text(delimiter="; ")[:50],
                    "Scores": str(pred.tolist()),
                }
            )

Predicted	Actual	Page ID	Text	Scores


  0%|          | 1/1194.6875 [00:04<1:37:33,  4.90s/batch]

END	IN	NL-HaNA_1.04.02_3712_0150.jpg	de waarde, van het voorwerp der bescher„; ming, de	[5.863048045284813e-06, 0.059085000306367874, 0.9409074783325195, 1.673855308581551e-06]


  0%|          | 2/1194.6875 [00:08<1:23:57,  4.22s/batch]

BEGIN	BEGIN	NL-HaNA_1.04.02_1082_0287.jpg	Het onse brieven vanden 28en december, 1623: p d'e	[0.9999990463256836, 7.214325705717783e-07, 2.554999980475259e-07, 6.420295961540035e-10]
END	IN	NL-HaNA_1.04.02_1816_0337.jpg	off men haer behoorl: rang wel; soude observeren, 	[3.417672996874899e-05, 0.007159017026424408, 0.9922771453857422, 0.0005295872106216848]


...

  3%|▎         | 30/1194.6875 [01:37<1:00:19,  3.11s/batch]

BEGIN	IN	NL-HaNA_1.04.02_1238_0459.jpg	weder te rugh gecomen de Commandeur; ijsbrandt God	[0.9905852675437927, 0.004420632496476173, 0.004910458344966173, 8.366243855562061e-05]
END	IN	NL-HaNA_1.04.02_7536_0295.jpg	op vervolgens schriftelijk berigt is; het daer op 	[0.00012508872896432877, 0.25609561800956726, 0.7418179512023926, 0.0019613259937614202]


  3%|▎         | 31/1194.6875 [01:40<1:00:59,  3.15s/batch]

END	IN	NL-HaNA_1.04.02_2480_0053.jpg	bewesen, met afsmeking; van sijne dierbare genade;	[9.449372555536684e-07, 1.37597213623053e-08, 0.9999984502792358, 5.828183589073888e-07]
BEGIN	IN	NL-HaNA_1.04.02_1653_0085.jpg	Watter op VEd:e Hoog agtb:; Eijs van 250 a 300000 	[0.9998195767402649, 9.451647929381579e-05, 8.532209903933108e-05, 5.58678834750026e-07]
BEGIN	IN	NL-HaNA_1.04.02_7532_0612.jpg	en die vande generale Jaarlijxe; verpagtinge voor 	[0.9982746839523315, 0.0016700949054211378, 5.302402496454306e-05, 2.0991035398765234e-06]
BEGIN	IN	NL-HaNA_1.04.02_3280_0505.jpg	te rug gezonden waaren, en de overige; in 't hospi	[0.9999629259109497, 1.1913843081856612e-05, 2.5062397980946116e-05, 6.872914326550017e-08]


  3%|▎         | 32/1194.6875 [01:41<1:01:21,  3.17s/batch]


In [21]:
writer = csv.DictWriter(
    sys.stdout,
    fieldnames=["Metric", "Average"] + [label.name for label in Label],
    delimiter="\t",
)
writer.writeheader()

for metric in (precision, recall, f1_score):
    scores = {
        label.name: f"{score:.4f}"
        for label, score in zip(Label, metric.compute().tolist())
    }
    writer.writerow(
        {"Metric": metric.__class__.__name__, "Average": str(metric.average)} | scores
    )

print(f"Accuracy ({accuracy.average} average):\t{accuracy.compute().item():.4f}")

Metric	Average	BEGIN	IN	END	OUT
MulticlassPrecision	None	0.1379	0.9989	0.1562	0.0000
MulticlassRecall	None	0.8000	0.9475	1.0000	0.0000
MulticlassF1Score	None	0.2353	0.9725	0.2703	0.0000
Accuracy (micro average):	0.9470
